# Feature Importance

Determine which features contribute the most to the output and see how well this coorelates with the COSMIC cancer gene census:

http://cancer.sanger.ac.uk/census/

Resources

https://en.wikipedia.org/wiki/Feature_selection#Wrapper_method

https://stats.stackexchange.com/questions/250381/feature-selection-using-deep-learning

http://blog.datadive.net/selecting-good-features-part-i-univariate-selection/


In [68]:
import numpy as np
import pandas as pd

In [67]:
# Load the model from disk as trained on the GPU box
from keras.models import model_from_json

print("Loading model...")
with open("models/model.json", "r") as f:
    model = model_from_json(f.read())
print("Loading weights...")
model.load_weights("models/weights.h5")
print("Compliling model...")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loading model...
Loading weights...
Compliling model...


In [109]:
"""
First do the simplest thing - sum the weights at the first layer by gene, sort,
and see if those with the highest weights intersect with the COSMIC list

Maybe it should be the sum of the absolute value? Negative or positive weight implies affect...
"""
weights = model.layers[2].get_weights()
ranks = np.absolute(np.sum(weights[0], axis=1))
rankings = pd.DataFrame(ranks, index=genes.astype('U')).sort_values(by=0, ascending=False)
rankings.head()

,0
RP11-78A18,23.470675
CTC-305H11,22.292587
LINC00363,19.987692
CTD-3096P4,19.076891
RP11-297L1,18.707317


In [110]:
# Load the features into a dataframe
import h5py
with h5py.File("data/tumor_normal.h5", "r") as f:
    genes = f["genes"][:]

In [112]:
rankings.index.values

array(['RP11-78A18', 'CTC-305H11', 'LINC00363', ..., 'MIR4745', 'GADL1',
       'ARL14EPL'], dtype=object)

In [113]:
cosmic = pd.read_table("cancer_genes.tsv")["Gene Symbol"].values

In [116]:
np.intersect1d(rankings.index.values[0:10000], cosmic).shape

(78,)